In [8]:
import torch.nn as nn
from math import *
import numpy as np
import torch
num_channels = 1 #Depending if its multimodal or just fMRI 

class AlexNet3D_Dropout(nn.Module): #AlexNet3D_Deeper_Dropout
    def __init__(self, num_classes=2):  #This is dependent on three way = 3, two way = 2 and regression = 1
        super(AlexNet3D_Dropout, self).__init__() #ModuleList
        
        self.features = nn.Sequential(        
            nn.Conv3d(num_channels, 64*num_channels, kernel_size=5,
                      stride=2, padding=0, groups=num_channels), #kernel 5 and padding 0 for uni, kernel 3 and padding 1 for multi
            nn.BatchNorm3d(64*num_channels),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=3, stride=3),

            nn.Conv3d(64*num_channels, 128*num_channels, kernel_size=3,
                      stride=1, padding=0, groups=num_channels),
            nn.BatchNorm3d(128*num_channels),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=3, stride=3),

            nn.Conv3d(128*num_channels, 192*num_channels,
                      kernel_size=3, padding=1, groups=num_channels),
            nn.BatchNorm3d(192*num_channels),
            nn.ReLU(inplace=True),

            nn.Conv3d(192*num_channels, 384*num_channels,
                      kernel_size=3, padding=1, groups=num_channels),
            nn.BatchNorm3d(384*num_channels),
            nn.ReLU(inplace=True),

            nn.Conv3d(384*num_channels, 256*num_channels,
                      kernel_size=3, padding=1, groups=num_channels),
            nn.BatchNorm3d(256*num_channels),
            nn.ReLU(inplace=True),

            nn.Conv3d(256*num_channels, 256*num_channels,
                      kernel_size=3, padding=1, groups=num_channels),
            nn.BatchNorm3d(256*num_channels),
            nn.ReLU(inplace=True),
            #nn.MaxPool3d(kernel_size=3, stride=3)
            nn.AdaptiveAvgPool3d([1,1,1]),
        )
        self.classifier = nn.Sequential(nn.Dropout(),
                                        nn.Linear(256*num_channels, 64),
                                        nn.ReLU(inplace=True),
                                        nn.Dropout(),
                                        nn.Linear(64, num_classes),
                                        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()


    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x  

'''    def forward(self, x):
        xp = self.features(x)
        for i, layer in enumerate(self.features):
            x = layer(x)
            print (i, x.size())
        x = xp.view(xp.size(0), -1)
        #print("final, ", x.shape)
        x = self.classifier(x)
        return [x, xp]'''

'''    def forward(self, x):
        layer_names = []
        print(x.shape)
        #x = self.features(x)
        layer = self.features[0]
        hold = layer(x)
        hold = hold.detach().cpu().numpy()
        np.save("layer_1",hold[0])
        for i, layer in enumerate(self.features):
            x = layer(x)
            layer_name = f"layer_{i}"
            print(f"{layer_name}: {x.size()}")
            layer_names.append(layer_name)
        x = x.view(x.size(0), -1)
        print("final:", x.shape)
        x = self.classifier(x)
        return [x]'''

          




'    def forward(self, x):\n        layer_names = []\n        print(x.shape)\n        #x = self.features(x)\n        layer = self.features[0]\n        hold = layer(x)\n        hold = hold.detach().cpu().numpy()\n        np.save("layer_1",hold[0])\n        for i, layer in enumerate(self.features):\n            x = layer(x)\n            layer_name = f"layer_{i}"\n            print(f"{layer_name}: {x.size()}")\n            layer_names.append(layer_name)\n        x = x.view(x.size(0), -1)\n        print("final:", x.shape)\n        x = self.classifier(x)\n        return [x]'

In [13]:
import torch
from torchsummary import summary

# Define a function to print the input shape
def print_input_shape(layer, input_size):
    print(f"Layer: {layer.__class__.__name__}")
    print(f"Input shape: {input_size}")
    return input_size

# Initialize your model
model = AlexNet3D_Dropout()

# Define the input shape
input_shape = (num_channels, 53, 63, 52)

# Print the input shape for each layer in the model
input_size = input_shape
for layer in model.children():
    input_size = print_input_shape(layer, input_size)

# Create a summary of the model
summary(model, input_shape, device="cpu")


Layer: Sequential
Input shape: (1, 53, 63, 52)
Layer: Sequential
Input shape: (1, 53, 63, 52)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 64, 25, 30, 24]           8,064
       BatchNorm3d-2       [-1, 64, 25, 30, 24]             128
              ReLU-3       [-1, 64, 25, 30, 24]               0
         MaxPool3d-4         [-1, 64, 8, 10, 8]               0
            Conv3d-5         [-1, 128, 6, 8, 6]         221,312
       BatchNorm3d-6         [-1, 128, 6, 8, 6]             256
              ReLU-7         [-1, 128, 6, 8, 6]               0
         MaxPool3d-8         [-1, 128, 2, 2, 2]               0
            Conv3d-9         [-1, 192, 2, 2, 2]         663,744
      BatchNorm3d-10         [-1, 192, 2, 2, 2]             384
             ReLU-11         [-1, 192, 2, 2, 2]               0
           Conv3d-12         [-1, 384, 2, 2, 2]       1,991,040
      Bat

In [9]:
import torch
from torchsummary import summary
#from your_model_file import AlexNet3D_Dropout  # Import your model class

# Initialize your model
model = AlexNet3D_Dropout()

# Create a summary of the model, specifying the input shape
summary(model, (num_channels, 53, 63, 52), device="cpu")

# Replace the values of input_depth, input_height, and input_width with your data dimensions


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 64, 25, 30, 24]           8,064
       BatchNorm3d-2       [-1, 64, 25, 30, 24]             128
              ReLU-3       [-1, 64, 25, 30, 24]               0
         MaxPool3d-4         [-1, 64, 8, 10, 8]               0
            Conv3d-5         [-1, 128, 6, 8, 6]         221,312
       BatchNorm3d-6         [-1, 128, 6, 8, 6]             256
              ReLU-7         [-1, 128, 6, 8, 6]               0
         MaxPool3d-8         [-1, 128, 2, 2, 2]               0
            Conv3d-9         [-1, 192, 2, 2, 2]         663,744
      BatchNorm3d-10         [-1, 192, 2, 2, 2]             384
             ReLU-11         [-1, 192, 2, 2, 2]               0
           Conv3d-12         [-1, 384, 2, 2, 2]       1,991,040
      BatchNorm3d-13         [-1, 384, 2, 2, 2]             768
             ReLU-14         [-1, 384, 

In [ ]:
import torch
import torchviz
#from your_model_file import AlexNet3D_Dropout  # Import your model class

# Initialize your model
model = AlexNet3D_Dropout()

# Create a random input tensor to trace the model
sample_input = torch.randn((1, 1, 53, 63, 52))

# Trace the model using torchviz
model_trace = torch.jit.trace(model, sample_input)

# Create a visualization of the model
dot = torchviz.make_dot(model_trace(sample_input))
dot.format = 'png'
dot.render("model_visualization")  # Save the visualization as a PNG file

# Replace the values of num_channels, input_depth, input_height, and input_width with your data dimensions


In [1]:
import torch.nn as nn
from math import *
import torch
from tensorboardX import SummaryWriter

num_channels = 1 #Depending if its multimodal or just fMRI 

class AlexNet3D_Dropout(nn.Module): #AlexNet3D_Deeper_Dropout
    def __init__(self, num_classes=2):  #This is dependent on three way = 3, two way = 2 and regression = 1
        super(AlexNet3D_Dropout, self).__init__() #ModuleList
        
        self.features = nn.Sequential(        
            nn.Conv3d(num_channels, 64*num_channels, kernel_size=5,
                      stride=2, padding=0, groups=num_channels), #kernel 5 and padding 0 for uni, kernel 3 and padding 1 for multi
            nn.BatchNorm3d(64*num_channels),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=3, stride=3),

            nn.Conv3d(64*num_channels, 128*num_channels, kernel_size=3,
                      stride=1, padding=0, groups=num_channels),
            nn.BatchNorm3d(128*num_channels),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=3, stride=3),

            nn.Conv3d(128*num_channels, 192*num_channels,
                      kernel_size=3, padding=1, groups=num_channels),
            nn.BatchNorm3d(192*num_channels),
            nn.ReLU(inplace=True),

            nn.Conv3d(192*num_channels, 384*num_channels,
                      kernel_size=3, padding=1, groups=num_channels),
            nn.BatchNorm3d(384*num_channels),
            nn.ReLU(inplace=True),

            nn.Conv3d(384*num_channels, 256*num_channels,
                      kernel_size=3, padding=1, groups=num_channels),
            nn.BatchNorm3d(256*num_channels),
            nn.ReLU(inplace=True),

            nn.Conv3d(256*num_channels, 256*num_channels,
                      kernel_size=3, padding=1, groups=num_channels),
            nn.BatchNorm3d(256*num_channels),
            nn.ReLU(inplace=True),
            #nn.MaxPool3d(kernel_size=3, stride=3)
            nn.AdaptiveAvgPool3d([1,1,1]),
        )
        self.classifier = nn.Sequential(nn.Dropout(),
                                        nn.Linear(256*num_channels, 64),
                                        nn.ReLU(inplace=True),
                                        nn.Dropout(),
                                        nn.Linear(64, num_classes),
                                        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()


    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x  


/data/users2/vitkyal/softwares/miniconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from tensorboardX import SummaryWriter
import torch

# Initialize the SummaryWriter
writer = SummaryWriter()

# Import your model (e.g., AlexNet3D_Dropout) and create an instance
model = AlexNet3D_Dropout()

# Create a dummy input (replace this with your actual input)
sample_input = torch.randn((1, num_channels, 53, 63, 52))

# Add the model graph to TensorBoard
writer.add_graph(model, sample_input)

AttributeError: module 'distutils' has no attribute 'version'